# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [2]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [3]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

/home/workspace
['/home/workspace/event_data/2018-11-27-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-07-events.csv', '/home/workspace/event_data/2018-11-09-events.csv', '/home/workspace/event_data/2018-11-19-events.csv', '/home/workspace/event_data/2018-11-05-events.csv', '/home/workspace/event_data/2018-11-22-events.csv', '/home/workspace/event_data/2018-11-16-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-24-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-20-events.csv', '/home/workspace/event_data/2018-11-06-events.csv', '/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-21-events.csv', '/home/workspace/event_data/2018-11-10-events.csv', '/home/workspace/event_data/2018-11-23-events.csv', '/home/workspace/event_data/2018-11-02-events.c

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


228


In [5]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

178


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [6]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [7]:
#  Create a Keyspace 
try:
    session.execute("create keyspace if not exists sparkifydatabase with replication = {'class':'SimpleStrategy','replication_factor':1}")
except Exception as e:
    print(e)

#### Set Keyspace

In [8]:
#  Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkifydatabase')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [9]:
##  Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
song_details1="create table if not exists song_details(artist text, songtitle text, song_length float, session_id int, iteminsession int, primary key (session_id,iteminsession))"
try:
    session.execute(song_details1)
except Exception as e:
    print(e)
                                                    

In [10]:
# We have provided part of the code to set up the CSV file#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO song_details(session_id, iteminsession, artist, songtitle, song_length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

In [11]:
#session.execute("drop table artist_details")

#### Do a SELECT to verify that the data have been inserted into each table


In [14]:
query_song_details="select artist,songtitle,song_length from song_details where session_id=%s and iteminsession=%s"
try:
    rows= session.execute(query_song_details, (139, 1))
    
except Exception as e:
    print(e)  
for row in rows:
    print("Artist: "+row.artist, "\nSong Title: "+row.songtitle, "\nSong Length: "+str(row.song_length))
    
    

Artist: Des'ree 
Song Title: You Gotta Be 
Song Length: 246.30812072753906


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [15]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
artist_details1="create table if not exists artist_details(artist text, song_title text, first_name text, last_name text, session_id int, iteminsession int, user_id int, primary key (user_id, session_id,iteminsession))"
try:
    session.execute(artist_details1)
except Exception as e:
    print(e)
                 


In [16]:
# We have provided part of the code to set up the CSV file#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO artist_details(user_id, session_id, iteminsession, artist, song_title, first_name, last_name)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))
                    

In [17]:
query_artist_details="select artist, song_title, first_name, last_name from artist_details where user_id = %s and session_id= %s "
try:
    rows= session.execute(query_artist_details, (10, 345))
    
except Exception as e:
    print(e)  
for row in rows:
    print("Artist: "+row.artist, "\nSong Title: "+row.song_title, "\nfirst_name: "+row.first_name, "\nlast_name: "+row.last_name)
    

Artist: Pavement 
Song Title: Mercy:The Laundromat 
first_name: Sylvie 
last_name: Cruz


In [18]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

user_details1="create table if not exists user_details(user_id int, song_title text, user_first_name text, user_last_name text, primary key ( user_id,song_title ))"
try:
    session.execute(user_details1)
except Exception as e:
    print(e)
                 
                    

In [19]:
#session.execute("drop table user_details")

In [20]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO user_details(user_id, song_title, user_first_name, user_last_name)"
        query = query + "VALUES (%s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[10]), line[9], line[1], line[4]))
                    

In [21]:
query_user_details="select song_title, user_first_name, user_last_name from user_details where user_id = %s and song_title = %s  "
try:
    rows= session.execute(query_user_details,(29,'Boom'))
    
except Exception as e:
    print(e)  
for row in rows:
    print("user firstname: "+row.user_first_name, "\nuser lastname: "+row.user_last_name)
    

user firstname: Jacqueline 
user lastname: Lynch


### Drop the tables before closing out the sessions

In [22]:
## Drop the table before closing out the sessions

In [23]:
session.execute("drop table song_details")
session.execute("drop table artist_details")
session.execute("drop table user_details")

### Close the session and cluster connection¶

In [24]:
session.shutdown()
cluster.shutdown()